In [136]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import urllib.request
import os
from pyquery import PyQuery as pq
import pickle
from selenium import webdriver
import datetime as dt
import time
import calendar
def get_homepage(i):
    url="https://etfdb.com/etf/"+all_ETF[i]
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    span=soup.find_all("a",text='Home page')
    url_homepage=span[0].get("href")
    return url_homepage
def get_the_last_day_of_month(start_date,end_date):
    ans = []
    temp = start_date - dt.timedelta(1)
    year = start_date.year
    month = start_date.month
    while (temp < end_date):
        temp = temp + dt.timedelta(calendar.monthrange(year,month)[1])
        ans.append(temp)
        month += 1
        if(month > 12):
            month = 1
            year += 1
    for i in range(len(ans)):
        if ans[i].isoweekday()>5:
            ans[i]=ans[i]+dt.timedelta(days=5-ans[i].isoweekday())
    return ans
last_day_of_month = get_the_last_day_of_month(dt.datetime(2016,1,1,0,0),dt.datetime(2018,12,31,0,0))

In [2]:
#將太晚發行ETF拿掉
origin_list=pd.read_excel("origin_list.xlsm",index_col=22,parse_dates=True)
need_list=origin_list.sort_index().loc[:"2016-01-01"]
need_list.to_excel("new_list.xlsx")

In [130]:
need_list=pd.read_excel("new_list.xlsx")
symbol_list = need_list['Symbol'].reset_index()['Symbol']
name_list = need_list['ETF Name'].reset_index()['ETF Name']
all_ETF=need_list["Symbol"].values
all_ETF

array(['EWM', 'ADRE', 'EEM', 'FXI', 'PGJ', 'VWO', 'EEV', 'EUM', 'SPEM',
       'GXC', 'GMF', 'FXP', 'DEM', 'DGS', 'FNI', 'AIA', 'PIE', 'HAO',
       'EPI', 'THD', 'PIN', 'AAXJ', 'EWX', 'EDC', 'EDZ', 'IDX', 'EET',
       'XPP', 'YINN', 'YANG', 'CQQQ', 'CHIC', 'CHIX', 'YAO', 'VNM', 'INDY',
       'CHII', 'CHIQ', 'CHIE', 'YXI', 'EIDO', 'SCIN', 'SCIF', 'ECNS',
       'EPHE', 'PEK', 'INDL', 'INXX', 'CHIM', 'ASEA', 'EDIV', 'MCHI',
       'FEM', 'FCA', 'DBEM', 'INCO', 'EEMV', 'INDA', 'NFTY', 'FEMS',
       'SMIN', 'EEMA', 'CXSE', 'DGRE', 'ASHR', 'KFYP', 'KWEB', 'AXJV',
       'KBA', 'CN', 'ASHS', 'CNXT', 'EMQQ', 'XCEM', 'CHAU', 'CHAD', 'EMGF',
       'ASHX', 'CNHX', 'XINA', 'KALL', 'AFTY'], dtype=object)

In [133]:
"""
homepage_list = []
for i in symbol_list:
    url = 'https://www.etf.com/'+i+'#overview'
    res = requests.get(url)
    doc = pq(res.text)
    doc.make_links_absolute(res.url)
    homepage_list.append(doc('.field-content.helplink')('a').attr('href'))
dic = {'symbol':[],'name':[],'addr':[]}
for i,s in enumerate(symbol_list):
    dic['symbol'].append(s)
    dic['name'].append(name_list[i])
    dic['addr'].append(homepage_list[i])
    print(s,name_list[i],homepage_list[i])
    
 # Save this dic
with open("ETF_dic.pkl","wb") as f:
    pickle.dump(dic,f)
"""

'\nhomepage_list = []\nfor i in symbol_list:\n    url = \'https://www.etf.com/\'+i+\'#overview\'\n    res = requests.get(url)\n    doc = pq(res.text)\n    doc.make_links_absolute(res.url)\n    homepage_list.append(doc(\'.field-content.helplink\')(\'a\').attr(\'href\'))\ndic = {\'symbol\':[],\'name\':[],\'addr\':[]}\nfor i,s in enumerate(symbol_list):\n    dic[\'symbol\'].append(s)\n    dic[\'name\'].append(name_list[i])\n    dic[\'addr\'].append(homepage_list[i])\n    print(s,name_list[i],homepage_list[i])\n    \n # Save this dic\nwith open("ETF_dic.pkl","wb") as f:\n    pickle.dump(dic,f)\n'

In [134]:
with open('ETF_dic.pkl','rb') as f:
    dic = pickle.load(f)

In [44]:
#print all ETF address
for i in range(len(all_ETF)):
    try:
        url="https://etfdb.com/etf/"+all_ETF[i]
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, 'html.parser')
        span=soup.find_all("a",text='Home page')
        url_homepage=span[0].get("href")
        print (url_homepage)
    except:
        print(all_ETF[i])     

http://us.ishares.com/product_info/fund/overview/EWM.htm?qt=EWM
https://www.invesco.com/portal/site/us/financial-professional/etfs/product-detail?productId=ADRE
http://us.ishares.com/product_info/fund/overview/EEM.htm?qt=EEM
http://us.ishares.com/product_info/fund/overview/FXI.htm?qt=FXI
http://invescopowershares.com/products/overview.aspx?ticker=PGJ
https://personal.vanguard.com/us/funds/snapshot?FundId=0964&FundIntExt=INT
http://www.proshares.com/funds/eev.html
http://www.proshares.com/funds/eum.html
https://us.spdrs.com/en/etf/spdr-portfolio-emerging-markets-etf-SPEM
https://www.spdrs.com/product/fund.seam?ticker=GXC
https://www.spdrs.com/product/fund.seam?ticker=GMF
http://www.proshares.com/funds/fxp.html
http://www.wisdomtree.com/etfs/fund-details.asp?etfid=50
http://www.wisdomtree.com/etfs/fund-details.asp?etfid=53
http://www.ftportfolios.com/retail/ETF/ETFsummary.aspx?Ticker=FNI
http://us.ishares.com/product_info/fund/overview/AIA.htm?qt=AIA
http://invescopowershares.com/product

In [33]:
# print unsolve website
for i in range(len(all_ETF)):
    try:
        url="https://etfdb.com/etf/"+all_ETF[i]
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, 'html.parser')
        span=soup.find_all("a",text='Home page')
        url_homepage=span[0].get("href")
        if "ishares" in url_homepage:#處理ishare的ETF
            continue
        elif "invesco" in url_homepage:#處理invesco的ETF
            continue
        elif "vanguard" in url_homepage: #vanguard
            continue
        elif "proshares" in url_homepage: #proshares
            continue
        elif "spdrs" in url_homepage: #spdrs
            continue
        elif "wisdomtree" in url_homepage: #wisdomtree
            continue
        elif "vaneck" in url_homepage:#vaneck
            continue
        elif "emerging" in url_homepage:#emerging
            continue
        elif "globalxfunds" in url_homepage:#globalxfunds
            continue
        else:
            print (url_homepage)
    except:
        print(all_ETF[i])
        

http://www.ftportfolios.com/retail/ETF/ETFsummary.aspx?Ticker=FNI
http://www.guggenheimfunds.com/etf/fund/HAO
http://direxionshares.com/etf/embu_bull_3x_shares.html
http://direxionshares.com/etf/embe_bear_3x_shares.html
http://www.direxioninvestments.com/products/direxion-daily-china-bull-3x-etf
http://www.direxioninvestments.com/products/direxion-daily-china-bull-3x-etf
http://guggenheiminvestments.com/products/etf/cqqq
CHIC
http://guggenheiminvestments.com/products/etf/yao
http://www.direxionshares.com/etf/indiabu_2x_shares.html
http://www.ftportfolios.com/Retail/etf/etfsummary.aspx?Ticker=FEM
http://www.ftportfolios.com/Retail/etf/etfsummary.aspx?Ticker=FCA
https://etfus.deutscheawm.com/US/EN/Product-Detail-Page/DBEM
NFTY
http://www.ftportfolios.com/retail/etf/etfsummary.aspx?Ticker=FEMS
https://etfus.deutscheawm.com/US/EN/Product-Detail-Page/ASHR
http://kraneshares.com/kfyp/
http://kraneshares.com/kweb/
http://kraneshares.com/kba/
https://etfus.deutscheawm.com/US/EN/Product-Detail-

In [ ]:
for i in range(1):
    try:
        url_homepage=get_homepage(i)
        print (url_homepage)
        if "ishares" in url_homepage:#處理ishare的ETF
            download_ishare(i,url_homepage)
        if "invesco" in url_homepage:#處理invesco的ETF
            NAV=download_invesco(i,url_homepage)
        if "vanguard" in url_homepage: #vanguard
            NAV=download_vanguard(5,url_homepage)
        if "proshares" in url_homepage: #proshares
        if "spdrs" in url_homepage: #spdrs
            NAV=download_spdrs(i,url_homepage)
        if "wisdomtree" in url_homepage: #wisdomtree
            url="https://etfdb.com/etf/"+all_ETF[13]
            resp = requests.get(url)
            soup = BeautifulSoup(resp.text, 'html.parser')
            span=soup.find_all("a",text='Home page')
            url_homepage=span[0].get("href")
            resp = requests.get(url_homepage)
            soup = BeautifulSoup(resp.text, 'html.parser')
            link_address=soup.find_all("a",{"class":"text-link fund-modal-trigger"})[0].get("data-href")
            data=pd.read_html(link_address,index_col=0,parse_dates=True)
        if "vaneck" in url_homepage:#vaneck
        if "emerging" in url_homepage:#emerging
            NAV=download_emerging(i,url_homepage)
        if "globalxfunds" in url_homepage:#globalxfunds
            NAV=download_globalxfunds(i,url_homepage)
    except:
        print(all_ETF[i])

In [31]:
import io
file1=io.open("data/EWM.xls", "r", encoding="utf-8")
pd.read_excel("data/EWM.xls")

In [11]:
#ishare 未能讀檔
def download_ishare(i,url_homepage):
    resp = requests.get(url_homepage)
    soup = BeautifulSoup(resp.text, 'html.parser')
    excel_file=soup.find_all("a",{"class":"icon-xls-export"})[3].get("href")
    domain_url = "https://www.ishares.com"
    excel_file= domain_url+excel_file
    file_name="data/"+all_ETF[i]+".xls"
    urllib.request.urlretrieve(excel_file, file_name)

In [34]:
url_homepage=get_homepage(0)

In [35]:
url_homepage

'http://us.ishares.com/product_info/fund/overview/EWM.htm?qt=EWM'

In [13]:
download_ishare(0,url_homepage)

In [8]:
pd.read_excel("data/EWM.xls")

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbf\xef\xbb\xbf<?'

In [115]:
#invesco
def download_invesco(i,url_homepage):
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1/data'}
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url_homepage)
    driver.find_element_by_link_text("Historical NAVs").click()
    time.sleep(3)
    driver.quit()
    testdir = '/Users/shayumin/Documents/文字探勘/homework1/data'
    list = os.listdir(testdir)
    list.sort(key=lambda fn:os.path.getmtime(testdir+'/'+fn))
    file_name=all_ETF[i]+".csv"
    os.rename(os.path.join(testdir,list[-1]),os.path.join(testdir,file_name))
    NAV=pd.read_csv(all_ETF[1]+".csv",index_col=2,parse_dates=True).loc[last_day_of_month]["NAV"]
    return NAV

In [119]:
testdir = '/Users/shayumin/Documents/文字探勘/homework1/data'
list = os.listdir(testdir)
list.sort(key=lambda fn:os.path.getmtime(testdir+'/'+fn))
list

['SCIN.xls',
 'EWM.xls',
 'CHIX.csv',
 'GXC.xls',
 'historical_navs_ADRE_1552630791359.csv',
 '.DS_Store']

In [117]:
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1/data'}
options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_options=options)
driver.get(url_homepage)
driver.find_element_by_link_text("Historical NAVs").click()
time.sleep(10)
driver.quit()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [88]:
#vanguard
def get_table(search_url,beginDate,endDate):
    driver = webdriver.Chrome()
    driver.get(search_url)
    driver.find_element_by_name("beginDate").send_keys(beginDate)
    driver.find_element_by_name("endDate").send_keys(endDate)
    driver.find_element_by_id("_cbdExt8").click()
    page_content=pd.read_html(driver.current_url)
    data_table=page_content[1].iloc[54:,:2]
    data_table=pd.DataFrame(data_table[1].values,index=pd.to_datetime(data_table[0]),columns=["Nav"])
    driver.quit()
    return  data_table
def download_vanguard(i,url_homepage):
    url_homepage=get_homepage(i)
    search_url="https://personal.vanguard.com/us/funds/tools/pricehistorysearch?"+url_homepage[url_homepage.find("FundId"):url_homepage.find("FundIntExt")-1]
    data_table_2016=get_table(search_url,"01/01/2016","12/31/2016")
    data_table_2017=get_table(search_url,"01/01/2017","12/31/2017")
    data_table_2018=get_table(search_url,"01/01/2018","12/31/2018")
    NAV=pd.concat([data_table_2016,data_table_2017,data_table_2018])
    return NAV["Nav"]

In [89]:
download_vanguard(5,url_homepage)

,Nav
0,
2016-01-04,$31.88
2016-01-05,$31.90
2016-01-06,$31.48
2016-01-07,$30.45
2016-01-08,$30.38
2016-01-11,$29.93
2016-01-12,$30.05
2016-01-13,$29.76
2016-01-14,$30.10


In [90]:
#proshares
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1'}
options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_options=options)
driver.get(url_homepage)
driver.find_element_by_link_text("NAV History").click()
time.sleep(3)
driver.quit()
testdir = '/Users/shayumin/Documents/文字探勘/homework1/data'
list = os.listdir(testdir)
list.sort(key=lambda fn:os.path.getmtime(testdir+'/'+fn))
filename=all_ETF[6]+'.csv'
os.rename(os.path.join(testdir,list[-1]),os.path.join(testdir,filename))

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"NAV History"}
  (Session info: chrome=72.0.3626.121)
  (Driver info: chromedriver=2.46.628411 (3324f4c8be9ff2f70a05a30ebc72ffb013e1a71e),platform=Mac OS X 10.13.6 x86_64)


In [53]:
#spdrs
def download_spdrs(i,url_homepage):
    domain_url = "https://us.spdrs.com/"
    excel_file= domain_url+"site-content/xls/"+all_ETF[9]+"_HistoricalNav.xls?fund="+all_ETF[9]+"&amp;docname=Most+Recent+NAV+%2F+NAV+History&amp;onyx_code1=&amp;onyx_code2="
    excel_file
    file_name="data/"+all_ETF[9]+".xls"
    urllib.request.urlretrieve(excel_file, file_name)
    file=pd.read_excel(file_name,index_col=0,header=2,parse_dates=True).dropna(axis=1,how="all")[:-12]
    file.index=pd.to_datetime(file.index)
    Nav=file["Nav"].loc[last_day_of_month]
    return(Nav)

In [54]:
download_spdrs(9,url_homepage)

Date
2016-01-29     63.970229
2016-02-29     62.850186
2016-03-31     69.972327
2016-04-29     69.901049
2016-05-31     69.180315
2016-06-30     69.573973
2016-07-29     71.731583
2016-08-31     77.079045
2016-09-30     78.847353
2016-10-31     77.367867
2016-11-30     76.766403
2016-12-30     72.733575
2017-01-31     77.501888
2017-02-28     80.460620
2017-03-31     81.953015
2017-04-28     83.737059
2017-05-31     87.703759
2017-06-30     89.357555
2017-07-31     96.852959
2017-08-31    100.574283
2017-09-29    101.905025
2017-10-31    105.652181
2017-11-30    106.644132
2017-12-29    107.374458
2018-01-31    120.650558
2018-02-28    113.057269
2018-03-30           NaN
2018-04-30    109.602897
2018-05-31    111.551643
2018-06-29    105.071446
2018-07-31    102.602652
2018-08-31     98.625406
2018-09-28     97.158569
2018-10-31     86.212438
2018-11-30     92.471344
2018-12-31     85.711104
Name: Nav, dtype: float64

In [31]:
#wisdomtree
url="https://etfdb.com/etf/"+all_ETF[13]
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')
span=soup.find_all("a",text='Home page')
url_homepage=span[0].get("href")
resp = requests.get(url_homepage)
soup = BeautifulSoup(resp.text, 'html.parser')
link_address=soup.find_all("a",{"class":"text-link fund-modal-trigger"})[0].get("data-href")
data=pd.read_html(link_address,index_col=0,parse_dates=True)

In [47]:
url_homepage[:25]

'http://www.wisdomtree.com'

In [9]:
#vaneck
url="https://etfdb.com/etf/"+all_ETF[25]
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')
span=soup.find_all("a",text='Home page')
url_homepage=span[0].get("href")
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1'}
options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_options=options)
url_link="https://www.vaneck.com/etf/equity/idx/documents/"
driver.get(url_homepage)
try:
    time.sleep(1)
    driver.find_element_by_link_text("Individual Investor").click()
    driver.find_element_by_link_text("I agree").click()
except:
    time.sleep(1)
time.sleep(3)
driver.find_element_by_link_text("Documents").click()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"Documents"}
  (Session info: chrome=72.0.3626.121)
  (Driver info: chromedriver=2.46.628411 (3324f4c8be9ff2f70a05a30ebc72ffb013e1a71e),platform=Mac OS X 10.13.6 x86_64)


In [5]:
url_homepage=get_homepage(41)

In [51]:
#emerging
def download_emerging(i,url_homepage):
    options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/shayumin/Documents/文字探勘/homework1/data'}
    options.add_experimental_option('prefs', prefs)
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url_homepage)
    driver.find_element_by_link_text("Download NAV/Premium Discount").click()
    time.sleep(40)
    driver.quit()
    testdir = '/Users/shayumin/Documents/文字探勘/homework1/data'
    list = os.listdir(testdir)
    list.sort(key=lambda fn:os.path.getmtime(testdir+'/'+fn))
    file_name=all_ETF[i]+".xls"
    os.rename(os.path.join(testdir,list[-1]),os.path.join(testdir,file_name))
    path="data/"+file_name
    day_NAV=pd.read_excel(path,index_col=0,header=4,parse_dates=True).loc[last_day_of_month]
    NAV=day_NAV[day_NAV["Fund Symbol"]==all_ETF[i]]["NAV"]
    return NAV

In [81]:
download_emerging(41,url_homepage)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [38]:
url_homepage=get_homepage(32)

In [29]:
url_homepage[:-4]+"fund/"+url_homepage[-4:]+"/?download_chart_data=true"

'http://www.globalxfunds.com/fund/CHIX/?download_chart_data=true'

In [52]:
#globalxfunds
def download_globalxfunds(i,url_homepage):
    ETF_len=len(all_ETF[i])
    download_link=url_homepage[:-ETF_len]+"fund/"+url_homepage[-ETF_len:]+"/?download_chart_data=true"
    file_name="data/"+all_ETF[i]+".csv"
    urllib.request.urlretrieve(download_link, file_name)
    NAV=pd.read_csv(file_name,index_col=0,header=1,parse_dates=True)["Closing Price"].loc[last_day_of_month]
    return NAV

In [53]:
download_globalxfunds(32,url_homepage)

Date
2016-01-29    11.64
2016-02-29    11.16
2016-03-31    12.49
2016-04-29    12.24
2016-05-31    12.11
2016-06-30    12.46
2016-07-29    12.89
2016-08-31    13.88
2016-09-30    14.46
2016-10-31    13.78
2016-11-30    14.15
2016-12-30    12.82
2017-01-31    13.42
2017-02-28     14.3
2017-03-31    14.48
2017-04-28    14.59
2017-05-31    15.34
2017-06-30    15.41
2017-07-31    16.72
2017-08-31    17.38
2017-09-29    16.95
2017-10-31    17.64
2017-11-30    17.82
2017-12-29    18.01
2018-01-31    20.51
2018-02-28    18.54
2018-03-30      NaN
2018-04-30    18.47
2018-05-31    18.08
2018-06-29    16.84
2018-07-31    17.32
2018-08-31    16.86
2018-09-28    16.99
2018-10-31    15.36
2018-11-30    16.45
2018-12-31    14.59
Name: Closing Price, dtype: object

In [61]:
url_homepage[:23]

'http://www.globalxfunds'

In [143]:
def check_download_file(com_data):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in dic['symbol']:
        data = com_data[s].values
        if (np.sum(np.isnan(data)) > 10):
            os.remove(f'./data/{s}.csv')
            get_single_symbol(s,browser,5)
    browser.quit()
def get_single_symbol(symbol,browser,wait = 3):
    
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')
    
    filepath = f'./data/{symbol}.csv'
    if os.path.isfile(filepath):
        print('File exist')
        return 
    
    url = f'https://finance.yahoo.com/quote/{symbol}/history?p={symbol}'
    browser.get(url)
    browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
    start_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px)']")
    start_date.clear()
    start_date.send_keys('1/1/2016')
    end_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px) Mstart(15px)']")
    end_date.clear()
    end_date.send_keys('1/1/2019')

    try:
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()
    except:
        browser.find_element_by_css_selector("[class=' Bd Bdc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) D(ib) C($c-fuji-blue-1-b) Bdc($actionBlueHover):h C($actionBlueHover):h Cur(p) Td(n)  Py(8px) cancel Miw(80px)! Fl(end)']").click()
        browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()

    browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Fl(end)']").click()
    time.sleep(5)
    browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
    time.sleep(wait)

    while not os.path.isfile(filepath):
        browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
        time.sleep(wait)

    print('Download',symbol)
def get_all_symbol_csv(dic):
    
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')

    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in dic['symbol']:
        get_single_symbol(s,browser)

    browser.quit()
def combine_data(dic,last_day_of_month):
    com_data = pd.DataFrame(last_day_of_month[::-1],columns=['Date'])
        
    for s in dic['symbol']:
        data = pd.read_csv(f'./data/{s}.csv')
        data['Date'] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
        
        # Mask
        mask = [False for i in range(data['Date'].shape[0])]
        for day in last_day_of_month:
            temp = data['Date'] == day
            mask |= temp
        data = data[mask]
        
        data_close = data[['Date','Close']].reset_index(drop=True)
        data_close = data_close.rename(columns={'Close':s})
        com_data = com_data.merge(data_close,on='Date',how='left')
        
    # check directory
    if not os.path.isdir('./Combine_data'):
        os.mkdir('./Combine_data')
    com_data.to_csv('./Combine_data/com_data.csv')
    
    return com_data
def check_download_file(com_data):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in dic['symbol']:
        data = com_data[s].values
        if (np.sum(np.isnan(data)) > 10):
            os.remove(f'./data/{s}.csv')
            get_single_symbol(s,browser,5)
    browser.quit()

In [145]:
get_all_symbol_csv(dic)

File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
File exist
Download KWEB
Download AXJV
Download KBA
Download CN
Download ASHS
Download CNXT
Download EMQQ
Download XCEM
Download CHAU
Download CHAD
Download EMGF
Download ASHX
Download CNHX
Download XINA
Download KALL
Download AFTY
